In [10]:
## NOTE: THis code may not be up to date -- the exact version is slightly different but has different filter conditions, 

from datetime import datetime
import networkx as nx
from collections import defaultdict
import sys
user_counts = defaultdict(lambda : 0)
biz_counts = defaultdict(lambda :0)

with open("user_business.csv") as f:
    for line in f:
        line = line.split("\n")[0].strip()
        u_id,b_id, date = line.split("\t")
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        user_counts[u_id] +=1
        biz_counts[b_id] += 1

user_business_interactions = []
with open("user_business.csv") as f:
    for line in f:
        line = line.split("\n")[0].strip()
        u_id,b_id, date = line.split("\t")
        date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
        if biz_counts[b_id] < 10:
            continue
        if user_counts[u_id] < 10:
            continue
        user_business_interactions.append((u_id, b_id, date))

user_business_interactions.sort(key = lambda x : x[2])



In [11]:
biz_cats = {}
with open("business_categories.csv") as f:
    lines = f.read().splitlines()
    for l in lines:
        bid, cat = l.split("\t")
        biz_cats[bid] = cat


In [12]:
# Define paramters for data extraction


slices_links = defaultdict(lambda : nx.MultiGraph())
slices_features = defaultdict(lambda : {})

START_DATE= datetime.strptime("2009-01-01", '%Y-%m-%d')
END_DATE= datetime.strptime("2016-11-01", '%Y-%m-%d')


SLICE_MONTHS = 6


In [13]:
all_nodes = []
for obj in user_business_interactions:
    u_id, b_id, date = obj
    if date < START_DATE:
        continue
    if date > END_DATE:
        break    
    all_nodes.append(u_id)
    all_nodes.append(b_id)
num_nodes = len (list(set(all_nodes)))
print ("total # nodes", num_nodes)
feat_dim = num_nodes

('total # nodes', 10435)


In [5]:


print ("Start date", START_DATE)

slice_id = 0
ctr = 0

for obj in user_business_interactions:
    u_id, b_id, date = obj
    #print (u_id, b_id, date)

    prev_slice_id = slice_id
    
    months_diff = (date-START_DATE).days/30.0
    
    if date > END_DATE:
        break
    if months_diff < 0:
        continue
        
    slice_id = int(months_diff // SLICE_MONTHS)
    
    if slice_id == 1+prev_slice_id and slice_id > 0:
        slices_links[slice_id] = nx.MultiGraph()
        slices_links[slice_id].add_nodes_from(slices_links[slice_id-1].nodes(data=True))
        # Empty feature vector to start for every time-slice ?
        
        slices_features[slice_id] = {}
        #for n in slices_links[slice_id].nodes():
        #    slices_features[slice_id][n] = np.zeros(feat_dim)

    if slice_id == 1+prev_slice_id and slice_id ==0:
        slices_links[slice_id] = nx.MultiGraph()
        #slices_author_features[slice_id] = {}

    if ctr % 10000 == 0:
        print (ctr)
    ctr +=1

    #if a1 not in slices_author_features[slice_id]:
    #    slices_author_features[slice_id][a1] = np.zeros(feat_dim)
    #slices_author_features[slice_id][a1] += feat
    if u_id not in slices_links[slice_id]:
        slices_links[slice_id].add_node(u_id, type='U')

    if b_id not in slices_links[slice_id]:
        slices_links[slice_id].add_node(b_id, type='B')
        
    slices_links[slice_id].add_edge(u_id, b_id, date=date)


Start date 2009-01-01 00:00:00
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [6]:
from scipy.sparse import csr_matrix
import scipy 
import numpy as np

for slice_id in slices_links:
    print ("# nodes in slice", slice_id, len(slices_links[slice_id].nodes()))
    print ("# edges in slice", slice_id, len(slices_links[slice_id].edges()))

    temp = np.identity(len(slices_links[max(slices_links.keys())].nodes()))
    
    slices_features[slice_id] = {}
    for idx, node in enumerate(slices_links[slice_id].nodes()):
        slices_features[slice_id][node] = temp[idx] 

# nodes in slice 0 1750
# edges in slice 0 3259
# nodes in slice 1 2598
# edges in slice 1 3438
# nodes in slice 2 3426
# edges in slice 2 4920
# nodes in slice 3 4165
# edges in slice 3 5433
# nodes in slice 4 5070
# edges in slice 4 7326
# nodes in slice 5 5833
# edges in slice 5 6935
# nodes in slice 6 6553
# edges in slice 6 7224
# nodes in slice 7 7265
# edges in slice 7 7177
# nodes in slice 8 7976
# edges in slice 8 8768
# nodes in slice 9 8764
# edges in slice 9 9580
# nodes in slice 10 9589
# edges in slice 10 11283
# nodes in slice 11 10419
# edges in slice 11 11396
# nodes in slice 12 11204
# edges in slice 12 12766
# nodes in slice 13 11814
# edges in slice 13 12807
# nodes in slice 14 12335
# edges in slice 14 13136
# nodes in slice 15 12728
# edges in slice 15 11474


In [7]:
# slices_labels 
slices_labels = defaultdict(lambda : {})
cats_list = list(set(biz_cats.values()))
cats_map = {k:v for k,v in zip(cats_list, range(0, len(cats_list)))}


date_delta = 3
for slice_id in slices_links:
    slice_nodes = slices_links[slice_id].nodes()
    slices_labels[slice_id] = []

    for b_id in slice_nodes:
        if b_id in biz_cats:
            label = biz_cats[b_id]
            slices_labels[slice_id].append((b_id, label))

    print ("# labels", len(slices_labels[slice_id]), "total # nodes", len(slice_nodes))



# labels 1131 total # nodes 1750
# labels 1614 total # nodes 2598
# labels 2037 total # nodes 3426
# labels 2373 total # nodes 4165
# labels 2747 total # nodes 5070
# labels 3017 total # nodes 5833
# labels 3299 total # nodes 6553
# labels 3553 total # nodes 7265
# labels 3823 total # nodes 7976
# labels 4079 total # nodes 8764
# labels 4333 total # nodes 9589
# labels 4561 total # nodes 10419
# labels 4837 total # nodes 11204
# labels 5044 total # nodes 11814
# labels 5273 total # nodes 12335
# labels 5460 total # nodes 12728


In [8]:

def remap(slices_graph, slices_features, slices_labels):
    all_nodes = []
    for slice_id in slices_graph:
        assert len(slices_graph[slice_id].nodes()) == len(slices_features[slice_id])
        all_nodes.extend(slices_graph[slice_id].nodes())
    all_nodes = list(set(all_nodes))
    print ("Total # nodes", len(all_nodes), "max idx", max(all_nodes))
    ctr = 0
    node_idx = {}
    idx_node = []
    for slice_id in slices_graph:
        for node in slices_graph[slice_id].nodes():
            if node not in node_idx:
                node_idx[node] = ctr
                idx_node.append(node)
                ctr += 1
    slices_graph_remap = []
    slices_features_remap = []
    for slice_id in slices_graph:
        G = nx.MultiGraph()
        for x in slices_graph[slice_id].nodes():
            G.add_node(node_idx[x])
        for x in slices_graph[slice_id].edges(data=True):
            G.add_edge(node_idx[x[0]], node_idx[x[1]], date=x[2]['date'])
        assert (len(G.nodes()) == len(slices_graph[slice_id].nodes()))
        assert (len(G.edges()) == len(slices_graph[slice_id].edges()))
        slices_graph_remap.append(G)
    for slice_id in slices_features:
        features_remap = []
        for x in slices_graph_remap[slice_id].nodes():
            features_remap.append(np.array(slices_features[slice_id][idx_node[x]]).flatten())
        features_remap = csr_matrix(np.squeeze(np.array(features_remap)))
        slices_features_remap.append(features_remap)
        print (slice_id, features_remap.shape)
    slices_labels_remap = []
    for slice_id in slices_labels:
        labels_remap = []
        for x,y in slices_labels[slice_id]:
            labels_remap.append((node_idx[x], y))
        slices_labels_remap.append(labels_remap)
    return (slices_graph_remap, slices_features_remap, slices_labels_remap)



In [9]:
slices_links_remap, slices_features_remap, slices_labels_remap = remap(slices_links, slices_features, slices_labels)


Total # nodes 12728 max idx zzF17hwmlFTuOa1Yagi-eg
1 (2598, 12728)
2 (3426, 12728)
3 (4165, 12728)
4 (5070, 12728)
5 (5833, 12728)
6 (6553, 12728)
7 (7265, 12728)
8 (7976, 12728)
9 (8764, 12728)
10 (9589, 12728)
11 (10419, 12728)
12 (11204, 12728)
13 (11814, 12728)
14 (12335, 12728)
15 (12728, 12728)
0 (1750, 12728)


In [99]:

np.savez('yelp/graphs.npz', graph=slices_links_remap)
np.savez('yelp/features.npz', feats=slices_features_remap)
np.savez('yelp/labels.npz', labels=slices_labels_remap)
